In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
# 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

In [3]:
# 카테고리 리스트
categories = {
    "스킨/토너": "100000100010013",
    "에센스/세럼/앰플": "100000100010014",
    "크림": "100000100010015"
}

In [4]:
def get_all_products(category_id):
    url = f"https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo={category_id}"
    driver.get(url)
    time.sleep(2)  # 로딩 대기
    
    # 무한 스크롤
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        # 상품 리스트 추출
    product_elements = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list li")
    product_links = []
    for elem in product_elements:
        try:
            link = elem.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            product_links.append(link)
        except:
            continue
    return product_links


In [5]:
def get_reviews(driver,product_url):
    driver.get(product_url)
    time.sleep(2)
    
    # 리뷰 탭 클릭
    try:
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "goods_reputation"))
        )
        review_tab.click()
    except:
        print(f"리뷰 탭 없음: {product_url}")
        return []

    time.sleep(2)
    reviews = []

    while True:
        time.sleep(1)
        review_items = driver.find_elements(By.CSS_SELECTOR, "#gdasList li")
        for item in review_items:
            try:
                text = item.find_element(By.CSS_SELECTOR, ".review_cont .txt_inner").text.strip()
                point = item.find_element(By.CSS_SELECTOR, ".review_cont .point").get_attribute("style")
                reviews.append({"text": text, "point": point})
            except:
                continue
                  # 다음 페이지 클릭
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, ".pageing a.next")
            next_button.click()
            time.sleep(2)
        except:
            break

    return reviews

In [6]:
# 누적할 리스트 생성 
all_reviews = [] 

In [ ]:
# 전체 실행
for category_name, category_id in categories.items():
    print(f"카테고리: {category_name}")
    product_links = get_all_products(category_id)
    print(f"상품 개수: {len(product_links)}개")

    for link in product_links:
        print(f"상품 링크: {link}")
        reviews = get_reviews(driver,link)
        print(f"리뷰 개수: {len(reviews)}")
        for review in reviews:
            print(f"점수: {review['point']}, 내용: {review['text']}")
            all_reviews.append({
                "카테고리": category_name,
                "상품URL": link,
                "리뷰점수": review['point'],
                "리뷰내용": review['text']
            })


카테고리: 스킨/토너
상품 개수: 20개
상품 링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000170266&dispCatNo=100000100010013&trackingCd=Cat100000100010013_Clickbest&curation=like&egcode=c001_c001&rccode=pc_category_01_a&egrankcode=2&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EB%A7%8E%EC%9D%B4%EB%B3%B8%EC%83%81%ED%92%88&t_number=1
리뷰 개수: 100
점수: width: 100%;, 내용: 성분이 좋습니다 악건성인 피부에 세안 마치자마자 열 번에서 20번 정도 펌핑해 주면서 피부에 흡수시키고 나면 건조함이 많이 사라집니다 계속 사용할 것 같습니다
점수: width: 100%;, 내용: 화정전이나 스킨케어시 사용하는데 가볍고 좋아요! 여름에 쓰기 좋을 듯
점수: width: 100%;, 내용: 가성비 좋은 토너에요
촉촉하고 맑아서 닥토로 쓰기 엄청 좋더라구요
몇통째 쓰고 있어요
점수: width: 100%;, 내용: 제가 월래 토너을 안쓰고 토너패드로 그냥 닥토로만 쓰는데 토너을 쓰고싶어 히알루론산이 건성에 좋다고하네요
이제품이 토리든 이제품이 왜 인기가 좋은지 일겠어요
물제형이고 손에 듬뿍 덜어 얼굴에 발라도 끈적거리지않고 여러번 덧발라도 잘흡수되요 저렵하게 잘산것같아요
점수: width: 100%;, 내용: 닦토나 스킨팩용으로 너무 좋구요. 수분감 유지도 좋아서 건성 피부에 사용하기 좋아요.
점수: width: 100%;, 내용: 용량도 넉넉하고 인지도 좋은 브랜드 제품이라서 선물하려고 구매했어요
점수: width: 100%;, 내용: 친구 선물로 줬어요. 속건조가 확실하게 잡혀서 쓰고 안쓰고의 차이가 확실히 난대요.
점수: width: 80%;, 

In [8]:
df = pd.DataFrame(all_reviews)
df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료: oliveyoung_reviews.csv")

CSV 저장 완료: oliveyoung_reviews.csv
